In [14]:
# import modules

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

import numpy as np

from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [15]:
# set device
device = torch.device("cuda:0")

# BERT 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /mnt/c/users/junzz/projects/capstone/simple-ntc/.cache/kobert_v1.zip


loading configuration file .cache/kobert_from_pretrained/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "return_dict": false,
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}

loading weights file .cache/kobert_from_pretrained/pytorch_model.bin


using cached model. /mnt/c/users/junzz/projects/capstone/simple-ntc/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at .cache/kobert_from_pretrained.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


In [26]:
# dataset

dataset_train = nlp.data.TSVDataset("./data/train.clean.tsv", field_indices=[1, 0])
dataset_test = nlp.data.TSVDataset("./data/test.clean.tsv", field_indices=[1, 0])

dataset_train[0]

['여러분제가 처음으로 스티커를 사보았습니다 오늘의 포스팅은 신나는 마음에 스티커를 남발할 수 있으니 이해 바래요 벌써남발중어제는 제가 여의도 숯닭에 다녀왔습니다다음날인 오늘부터 남편이 일주일간 출장을 가서 ㅠㅠ 마지막 만찬 느낌으로닭갈비 먹으러 갔네요 이 곳은 여의도에서 닭갈비 맛집으로 이름좀 날린 모양인데영등포시장역 인근으로이전했다고 합니다 영등포시장역 맛집 여의도 숯닭 야들야들한 닭갈비 화로에 올려 분위기 좋은 곳영등포시장역 번출구에서m 떨어져있으며아크로타워스퀘어 한림대학교 한강성심병원 근처에 있습니다본 건물주께서는 힘든 세입자를 위해월세 한달치를 감면해주셨습니다 요즘 코로나로 장사에 어려움이 있을텐데이런 착한 건물주들 칭찬합니다 바깥에서 메뉴판도 확인할 수 있고이벤트도 있네요산토리 하이볼 할인 EVENT원 > 원산토리 하이볼 잔 주문시김치전 or 감자전서비스◾매장내부◾사람이 좀 있길래 다 먹고 나갈 때 찍어야지 했는데나갈 때 되니 사람이 더 많아졌어요흑 ㅋㅋㅋㅋㅋ 사진은 찍어야 하니까얼른 찍고 나왔습니다도망 아님◾MENU◾ 메인메뉴 슻불닭갈비 더덕구이 생삼겹특제양념삼겹 껍데기 매운닭발 LA갈비 찌개메뉴 비지찌개 차돌된장 청국장 김치찌개 돼지고기 두부 짜글이 사이드메뉴 감자전 배추해물전 김치전 묵사발 계란찜 엉뚱한 골뱅이 쫄면 메뉴판을 정면으로 못찍어서 직접 적어드렸습니다 확대하면 보이긴 해요 새로 나온 메뉴들도 있나봐요 살치살 g을무려 에 준다고 해요 또 하나는 레몬 냉면이름만 들으면 신기한데 식초의 역할을 레몬이 대신해줘서 맛있을 것도 같고 궁금하네요자자주문을 해볼까요닭갈비 인분 주세요양념소금구이 닭갈비 중에 고를 수 있습니다우린 양념으로 주문◾ 숯불닭갈비 원 ◾고기를 주문하니화로를 이렇게 통채로 왠지 분위기 나고 좋은듯요밑반찬으로는 상추 깻잎 쌈채와 겉절이와 마늘 쌈장 심플하지만 요거요 겉절이깔끔하고 맛있습니다잠시 기다리니 닭갈비를 접시에 담아서 주셨는데먹기 좋게 다 잘라주셔서 화로에 올리기만 하면 된답니다처음에 초벌해준건가 하고 화로에 올려놓고익을

In [27]:
class BERTDataset(Dataset):

    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transformer = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair
        )
        self.sentences = [transformer([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [46]:
# SET Parameters

MAX_LEN = 510
BATCH_SIZE = 16
WARMUP_RATIO = 0.1
NUM_EPOCHS = 10
MAX_GRAD_NORM = 1
LOG_INTERVAL = 200
LEARNING_RATE = 5e-5

In [47]:
# tokenize
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, MAX_LEN, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, MAX_LEN, True, False)

using cached model. /mnt/c/users/junzz/projects/capstone/simple-ntc/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [48]:
train_dataloader = torch.utils.data.DataLoader(
    data_train, batch_size = BATCH_SIZE, num_workers = 4
)
test_dataloader = torch.utils.data.DataLoader(
    data_test, batch_size = BATCH_SIZE, num_workers = 4
)

In [49]:
class BERTClassifier(nn.Module):

    def __init__(self, 
                bert, 
                hidden_size = 768, 
                num_classes = 2, 
                dr_rate = None, 
                params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [50]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [51]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": 0.01},
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * WARMUP_RATIO)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [52]:
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc

In [53]:
# TRAINING

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    model.train()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % LOG_INTERVAL == 0:
            print("epoch {} batch id {} loss {} train acc {}"
                .format(e + 1, batch_id + 1, loss.data.cpu().numpy(), train_acc / (batch_id + 1)))
    
    print("epoch {} train acc {}".format(e + 1, train_acc / (batch_id + 1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

    print("epoch {} validation acc {}".format(e + 1, test_acc / (batch_id + 1)))
        

/home/timo/.pyenv/versions/3.7.12/envs/ml-3.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1020 [00:00<?, ?it/s]

RuntimeError: CUDA error: unknown error